# Parallel computing

In this example, we demonstrate how parallel computing - a method of performing multiple calculations simultaneously to improve efficiency - is applied to estimate the probability of levee failure due to wave overtopping. 

First, we import the necessary packages:

In [4]:
from probabilistic_library import ReliabilityProject, DistributionType, ReliabilityMethod
import time

We consider the limit state function for wave overtopping (which we have artificially slowed down):

In [5]:
from utils.models import ZFunctionOvertopping

And the following reliability project, in which we apply the `crude_monte_carlo` method:

In [6]:
def define_project():

    project = ReliabilityProject()
    project.model = ZFunctionOvertopping.z_sleep
    
    project.variables["h"].distribution = DistributionType.log_normal
    project.variables["h"].mean = 1.5
    project.variables["h"].deviation = 0.05

    project.variables["hm0"].distribution = DistributionType.log_normal
    project.variables["hm0"].mean = 1.5
    project.variables["hm0"].deviation = 0.25

    project.variables["tm10"].distribution = DistributionType.log_normal
    project.variables["tm10"].mean = 3
    project.variables["tm10"].deviation = 0.5

    project.variables["wave_direction"].distribution = DistributionType.deterministic
    project.variables["wave_direction"].mean = 0.0

    project.variables["dike_normal"].distribution = DistributionType.deterministic
    project.variables["dike_normal"].mean = 0.0

    project.variables["y_crest"].distribution = DistributionType.deterministic
    project.variables["y_crest"].mean = 6.0

    project.variables["q_crit"].distribution = DistributionType.log_normal
    project.variables["q_crit"].mean = 0.001
    project.variables["q_crit"].deviation = 0.01

    project.settings.reliability_method = ReliabilityMethod.crude_monte_carlo
    project.settings.minimum_samples = 1000
    project.settings.maximum_samples = 1000
    project.settings.variation_coefficient = 0.02

    return project

If not specified, the calculations are performed on a single processor. To utilize multiple processors, we adjust the setting `project.settings.max_parallel_processes`. With the following code, we analyze the effect of using multiple processors on the calculation time:

In [7]:
max_parallel_processes = [1, 4, 8]

for val in max_parallel_processes:

    project = define_project()
    project.settings.max_parallel_processes = val

    t = time.time()
    project.run()
    elapsed = time.time() - t

    print(f"Max parallel processes: {val}")
    print(f"Reliability index: {project.design_point.reliability_index}")
    print(f"Time elapsed: {elapsed} seconds")
    print(f"----------------------------------------------------------------")

Max parallel processes: 1
Reliability index: 1.2265281200366105
Time elapsed: 100.79533386230469 seconds
----------------------------------------------------------------
Max parallel processes: 4
Reliability index: 1.2265281200366105
Time elapsed: 25.988253355026245 seconds
----------------------------------------------------------------
Max parallel processes: 8
Reliability index: 1.2265281200366105
Time elapsed: 13.630357503890991 seconds
----------------------------------------------------------------
